# The Brusselator

The Brusselator is a model of hypothetical autocatalytic reactions in a diffusive medium. Nobel Prize winner Ilya Prigogine (1977) from University of Bruxelles proposed the model, hence the name.

The system of chemical reactions writes:
\begin{eqnarray}
A&\overset{k_1}{\longrightarrow}& U \\
2U + V&\overset{k_2}{\longrightarrow}& 3U \\
B+U&\overset{k_3}{\longrightarrow}& V+C \\
U&\overset{k_4}{\longrightarrow}& D
\end{eqnarray}

The concentrations $[A]$ and $[B]$ are assumed to be in large excess and therefore constant
The first step is the formation of an intermediate specie $U$, the second is the autocatalytic step for $U$ and the consumption of $V$, the third is the regeneration of $V and the generation of $C$ and the fourth the generation of D. Altogether, this system reduces to

$$
A+B\longrightarrow C+D
$$

Since $[A]$ and $[B]$ are constant, and $C$ and $D$ do not react with any other specie, only two equations describe the reaction process:
\begin{eqnarray}
\frac{d[U]}{d\tau}&=&k_1[A]+k_2[U]^2[V]-k_3[B][U]-k_4[U]\\
\frac{d[V]}{d\tau}&=&-k_2[U]^2[V]+k_3[B][U]
\end{eqnarray}
where $\tau$ is time.

Using the following non-dimensional variable,
\begin{equation}
u = [U]\sqrt{\frac{k_2}{k_4}};\;v=[V]\sqrt{\frac{k_2}{k_4}};\;a=[A]\frac{k_1}{k_4}\sqrt{\frac{k_2}{k_4}};\;b=[B]\frac{k_3}{k_4};\;t=k_4\tau\,,
\end{equation}
and adding the diffusion process, yields
\begin{eqnarray}
\frac{\partial u}{\partial t}&=&a+u^2v-(b+1)u+D_u\nabla^2u\\
\frac{\partial v}{\partial t}&=&-u^2v+bu+D_v\nabla^2v
\end{eqnarray}
where $D_u$ and $D_v$ are the normalized diffusion coefficients of species $U$ and $V$.

The Brusselator is solved over a domain of dimensions $L_x\times L_y=L^2$, with periodic boundary conditions on all edges. Periodicity is a mathematical tool used here to model a subset of a very large domain. Any variable $q$ defined over the domain satisfies, for $x\in[0,Lx]$ and $y\in[0,Ly]$,

\begin{equation}
q(x\pm mLx,y\pm nLy,t)=q(x,y,t)
\end{equation}

where $m$ and $n$ are any integer. For periodic or chaotic solutions, periodicity is acceptable if the domain is large enough to contain the most dynamically relevant scales. Spatial correlations should approach zero for separations approaching the shortest half length of the domain.

The variables $u$ and $v$ must be initialize in order to solve the governing equation above. The values chosen by the user may lead to different states, depending on the choice of $a$ and $b$. This problem falls in the category of initial value problem (IVP).

## Methods

### Time discretization
The most popular method for IVP is the 4<sup>th</sup> Runge-Kutta method (RK41). Given an IVP problem, 
$$
\frac{dy}{dt} = f(y,t)\;,
$$
starting at time $t=t_0$ and resolved with a time step $\Delta t$,
the RK41 integration from $u^{(n)}=y(t_n)$ to $u^{(n+1)}=y(t_{n+1})$ ($t_n = t_0+n\Delta t$) writes
$$
y^{(n+1)}=y^{(n)}+\frac{1}{6}k_1+\frac{1}{3}(k_2+k_3)+\frac{1}{6}k_4
$$
with
\begin{eqnarray}
k_1 &=& \Delta t\,f\left(y^{(n)},t_n\right)\\
k_2 &=& \Delta t\,f\left(y^{(n)}+\frac{1}{2}k_1,t_n+\frac{1}{2}\Delta t\right)\\
k_3 &=& \Delta t\,f\left(y^{(n)}+\frac{1}{2}k_2,t_n+\frac{1}{2}\Delta t\right)\\
k_4 &=& \Delta t\,f\left(y^{(n)}+k_3,t_n+\Delta t\right)
\end{eqnarray}

### Spatial discretization
Since the domain is square and periodic, the grid is uniform with $N$ grid steps of length $\Delta=L/N$. The variables $u$ and $v$ are stored at the cells' center $(x_i,y_i)=\left((i+1/2)\Delta,(j+1/2)\Delta\right)$ with $i$ and $j$ in the range $[0,N-1]$.

For $i$ and $j$ in the range $[1,N-2]$, the Laplacian in the diffusion term is discretized with a 2<sup>nd</sup>-order finite difference scheme,
$$
\nabla^2u^{(n)}_{i,j}=\frac{u^{(n)}_{i-1,j}+u^{(n)}_{i+1,j}+u^{(n)}_{i,j-1}+u^{(n)}_{i,j+1}-4u^{(n)}_{i,j}}{\Delta^2}
$$

At the boundaries, like $i=0,j\in[1,N-2])$, periodicity must be enforced as
$$
u^{(n)}_{-1,j} = u^{(n)}_{N-1,j}
$$
yielding
$$
\nabla^2u^{(n)}_{0,j}=\frac{u^{(n)}_{N-1,j}+u^{(n)}_{1,j}+u^{(n)}_{0,j-1}+u^{(n)}_{0,j+1}-4u^{(n)}_{0,j}}{\Delta^2}
$$

## Code
### Modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Parameters

In [ ]:
L = 200. # length of computational domain
N = 200 # Number of computational nodes per direction
Delta = L/N #Size of computational cell
x = np.linspace(Delta/2,L-Delta/2,N) # horizontal location of comp. nodes
z = np.linspace(Delta/2,L-Delta/2,N) # vertical location of comp. nodes
X,Z = np.meshgrid(x,z,indexing='ij')
a =4.5
b = 7.5
i_u = 0; i_v = 1 
D = np.array([2.,16.])
D_Deltasq = D/Delta**2
dt = 0.01 #time step
T = 50. #integration time
# initisl values (mean values)
u0 = 1.0
v0 = 1.0
# intensity of noise added to mean values
noisemag = 0.0001

### Definition of the RHS

In [ ]:
def rhs(y,t):
    global a,b,D_Deltasq
    f = np.zeros_like(y)
    # Reaction stuff
    f[:,:,i_u] = a + y[:,:,i_u]**2*y[:,:,i_v] - (b+1)*y[:,:,i_u]
    f[:,:,i_v] = -y[:,:,i_u]**2*y[:,:,i_v] + b*y[:,:,i_u]

    return f

### Time integration

In [ ]:
t = 0
y = np.zeros((N,N,2),dtype = 'float')
y[:,:,i_u] = np.random.normal(u0,noisemag*u0,(N,N))
y[:,:,i_v] = np.random.normal(u0,noisemag*v0,(N,N))

while t < T:


In [ ]:
plt.figure(figsize=(3,2.75),dpi = 300)
plt.contourf(X,Y,y[:,:,i_u])
plt.show()

In [ ]:
y[:,:,i_u]